In [1]:
!pip install dateparser xlsxwriter

     |████████████████████████████████| 368kB 4.5MB/s 
     |████████████████████████████████| 153kB 8.3MB/s 


In [2]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 11 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 0s (26.9 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5

In [3]:
import json
import os
import numpy as np
import pytesseract
from PIL import Image
from pytesseract import Output
import cv2
import pandas as pd
# from StyleFrame import StyleFrame
import re
from dateparser.search import search_dates
import pandas as pd
from glob import glob
from datetime import datetime
from PIL import Image
import argparse
import xlsxwriter


In [5]:
input_images_dir = r"/content/img"
inp_img_jsons_dir= r"/content/img"
output_excel_dir = r"/content/output"

In [6]:

def detectText(img_path,y1, x1, y2, x2):
    img = cv2.imread(img_path, 0)
    crop_img = img[int(y1): int(y2), int(x1): int(x2)]
    im = Image.fromarray(crop_img)
    im.save("/tmp/output.jpeg")

    BLACK=[255,255,255]
    try:
        resized_img = cv2.resize(crop_img ,(2*crop_img.shape[1], 2*crop_img.shape[0]), interpolation=cv2.INTER_CUBIC)
        #resized_img = crop_img
        #padding the image
        #pad_img = cv2.copyMakeBorder(resized_img, 15, 15,15,15, cv2.BORDER_CONSTANT, value=BLACK)

        #resized_img = cv2.resize(pad_img ,(2*pad_img.shape[0], 2*pad_img.shape[1]), interpolation=cv2.INTER_CUBIC)

        k_size = 1
        kernel = np.ones((k_size,k_size), np.uint8)

        #apply morphological operation to dilate and erode
        morph_img = cv2.morphologyEx(resized_img, cv2.MORPH_CLOSE, kernel,iterations=1)

        blur_img = cv2.threshold(cv2.bilateralFilter(morph_img, 2, 75, 75), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

        custom_oem_psm_config = r'--oem 1 --psm 6 '
        text = pytesseract.image_to_string(blur_img, lang="eng", config=custom_oem_psm_config)
        print(text+"\n")
    except: 
        raise
        raise Exception('NoCell', 'The given points are not valid.')
    return (blur_img, text)

In [7]:

def get_error_dataframes(comp_df):
    error_df = pd.DataFrame()
    empty_df=pd.DataFrame()
    emp_list=[]
    found = False
    found_error=False
    for col in comp_df:
        if comp_df[col]=="-" or comp_df[col]=="":
            found=True
            emp_list.append(col)
    val= ", ".join(emp_list)
    empty_df = pd.DataFrame([val])
    
    return (empty_df if found else None)


In [8]:
def writeToExcel(component_data, component_classes, img_path,f_type="single"):
    global all_invoices_data,count_of_processed_pdfs,writer,next_pdf_start_row,next_error_df_start_row
    
    filename = os.path.basename(img_path)
    filename = filename.rpartition('.')[0]
    #print("processed file: ",filename)
    try:
        #print(component_data)
        emp_row_df= pd.DataFrame().append(pd.Series(), ignore_index=True)
        #c_df = pd.concat([emp_row_df,component_data.T],ignore_index=True)
        c_df = component_data.T
        #print("Cdf",c_df)
        merged_df = pd.concat([pd.DataFrame([filename]),c_df],axis=1)
        merged_df= merged_df.append(pd.Series(), ignore_index=True)
        merged_df=merged_df.replace({'=':''}, regex=True)
        merged_df=merged_df.replace({'\n':''}, regex=True)
        merged_df.to_excel(writer,sheet_name='Invoice Data',index=False,header=False,startrow=next_pdf_start_row,encoding='utf-8')
        print(merged_df)        
        count_of_processed_pdfs+=1
        next_pdf_start_row+=merged_df.shape[0]+1
            
        (empty_df)=get_error_dataframes(component_data)
        print("Empty df",empty_df)
        if empty_df is not None:
            empty_df.to_excel(writer, sheet_name='Validation Failure',index = False,header=False, startrow=next_error_df_start_row, startcol=1)
    
        elif empty_df is not None and error_df is None:
            next_error_df_start_row+=empty_df.shape[0] 

        #print(count_of_processed_pdfs)    
        #print(filename,des_folder+"/"+filename+'.xlsx')                
        return ("Write to Excel Successful for image "+filename)
    except Exception as e:
        
        #traceback.print_exc()
        return ("Write to Excel not Successful for image "+filename)


In [9]:
def getComponentJsonData(data):
    #geeting the index of objects detected with scores more than 0.5
    index = len((np.array(data['detection_scores']) >= 0.5).nonzero()[0])
    detection_scores = np.array(data['detection_scores'][:index])
    detection_classes = np.array(details_dict[data['detection_classes'][:index]])
    detection_boxes = np.array(data['detection_boxes'][:index])
    image_width = np.array(data['image_size'][0])
    image_height = np.array(data['image_size'][1])
    
    y_min = np.around(detection_boxes[:,0]*image_height, 2)
    x_min = np.around(detection_boxes[:,1]*image_width, 2)
    y_max = np.around(detection_boxes[:,2]*image_height, 2)
    x_max = np.around(detection_boxes[:,3]*image_width, 2)

    return (y_min, x_min, y_max, x_max, detection_classes,detection_scores)


In [10]:

def get_component_df(component_data,image_file):    


    #getting dataframe for component
    try:
        (y_min1, x_min1, y_max1, x_max1, component_classes,detection_scores) = getComponentJsonData(component_data)
        
        component_dataframe = componentDetection(y_min1, x_min1, y_max1, x_max1, image_file, component_classes,detection_scores)
        
    except Exception as e: 
        print(e)
        traceback.print_exc()
        component_dataframe= pd.DataFrame(["No components found"])
        component_classes=[]
        
    return component_dataframe,component_classes


In [11]:
def write_excel_for_image(img_path): 
    json_path = os.path.join(inp_img_jsons_dir,os.path.basename(img_path)[:-4]+".json")
    write_status="Failed"
    with open(json_path) as f:
        comp_data = json.load(f)
        try:  
            component_dataframe,component_classes = get_component_df(comp_data,img_path)            
            write_status = writeToExcel(component_dataframe, component_classes, img_path)
            filename = os.path.basename(img_path)
            filename = filename.rpartition('.')[0]            
            #remove_img_jsons(filename)
            
        except Exception as e:
            #traceback.print_exc()
            if e.args[0] == 'NoCell':
                #print('Error: '+e.args[1])
                pass
            else:   
                pass             
    return write_status



In [12]:
def start():
    all_images = glob(input_images_dir+"/*.jpg")
    for img_path in all_images:
        write_excel_for_image(img_path)

def freeze_panes_and_apply_header_formats():
    worksheet = writer.sheets['Invoice Data']
    # Add a header format.
    header_format = workbook.add_format({
        'bold': True,
        'text_wrap': True,
        'valign': 'top',
        'fg_color': '#D7E4BC',
        'border': 1})
    try:    
        worksheet2 = writer.sheets['Validation Failure']
        df=pd.DataFrame(columns=["File Name","Missing Elements","Validation Failures","Value"])
        # Write the column headers with the defined format.
        for col_num, value in enumerate(df.columns.values):        
            worksheet2.write(0, col_num, value, header_format)
        worksheet2.freeze_panes(1, 0)
    except:
        pass
    df=pd.DataFrame(columns=["File Name","Invoice Number","Invoice Date","Vendor Name","Invoice Total"])
    # Write the column headers with the defined format.
    for col_num, value in enumerate(df.columns.values):       
        worksheet.write(0, col_num, value, header_format)
    worksheet.freeze_panes(1, 0)
    


In [13]:


#getting component text from tesseract
def componentDetection(y_min, x_min, y_max, x_max, img_path, detection_classes,detection_scores):
    bl=[]
    text_array=[]
    comp_dict = {"Invoice Number":"-","Invoice Date":"-","Vendor Name":"-","Invoice Total":"-"}
    for i in range(0,len(x_min)):
        (imgg,tess_text)=detectText(img_path,y_min[i],x_min[i], y_max[i], x_max[i])
        bl.append(imgg)
        
        #data.append(text1)
        #img_array.append(tess_img)
        if(detection_classes[i]=="vendor" ):
            ven_detail = re.sub("[^\w.,:\s]", '',tess_text.strip()) #|#*{<>}\\[\\]
            classname= "Vendor Name" if detection_classes[i]=="vendor" else "Vendor Address"
            comp_dict[classname]=ven_detail
            text_array.append(ven_detail)

        elif(detection_classes[i]=="inv_num"):             
            inv_regex = re.sub("[\W\s]", '',tess_text)
            #inv_regex = re.sub("[|#*{<&°>} « \\[ \\] « , ]", '',tess_text)
            inv_num=inv_regex
            inv_num = re.split(r'(\d+)', inv_regex, 1)
            inv_num = " ".join(inv_num[1:]) if len(inv_num)>1 else inv_num[0]
            comp_dict["Invoice Number"]=inv_num
            text_array.append(inv_num)

       # elif(detection_classes[i]=="tax"): 
            
       #     #trimmed_value = re.sub("[| \\[ \\] ]", '',tess_text)
       #     tax = tess_text.split(' ')[-1]
       #     trimmed_tax = re.sub("[\W]", '',tax)
       #     text_array.append(trimmed_tax)
       #     comp_dict["Tax"]=trimmed_tax
        elif(detection_classes[i]=="int_total"):
            #trimmed_value = re.sub("[|]", '',tess_text)
            values = tess_text.split(' ')[-1]
            tr= re.sub("[^\d.,$]", '',values)
            
            trimmed_value1=tr
            #trimmed_value1 = re.sub("[\W]", '',values) #"[a-zA-Z|#©§*{<:>}(&)\\[~«\\]-]", '',values)

            #classname= "Subtotal" if detection_classes[i]=="stotal" else "Invoice Total"
            classname= "Invoice Total"
            comp_dict[classname]=trimmed_value1
            text_array.append(trimmed_value1)
        else:
            dates = search_dates(tess_text)
            #print(dates)
            #text_array.append(dates)
            if dates!=None:
                text_array.append(dates[0][0])
                comp_dict["Invoice Date"]=dates[0][0]
            else:
                trimmed_date = re.sub("[a-zA-Z|©#§*{<:>}(&)\\[-~«\\]]", '',tess_text)
                #trimmed_date = re.sub("[\W]", '',tess_text)   
                comp_dict["Invoice Date"]=trimmed_date
                text_array.append(trimmed_date)

        #df1.update(text1)
   
    comp_df = pd.DataFrame.from_dict(comp_dict, orient="index")
    
    #comp_df.insert(0,"Component",detection_classes)
    
    return comp_df


In [14]:

error_dict = {}
all_invoices_data=pd.DataFrame().append(pd.Series(), ignore_index=True)
merge_cells_positions=[]
count_of_processed_pdfs=0
next_pdf_start_row=2
next_error_df_start_row=1

now = datetime.now()
all_inv_dt_string = now.strftime("%Y_%m_%d_%H")

#writer objects
actual_path = os.path.join(output_excel_dir,all_inv_dt_string+".xlsx")
duplicate_files_count=0
path = actual_path
while (os.path.isfile(path)):
    duplicate_files_count+=1
    path=actual_path[:-5]+"_"+str(duplicate_files_count) +".xlsx"

writer = pd.ExcelWriter(path, engine='xlsxwriter')
all_invoices_data.to_excel(writer,sheet_name='Invoice Data',index=False,header=False)
workbook  = writer.book

#creating dictionaries as per pbtxt
table_dict = np.array(['','table','row', 'column'])
#details_dict = np.array(['', 'inv_num','inv_date','inv_tot','stotal','vendor','addr','tax'])
details_dict = np.array(['', 'inv_num','inv_date','vendor','int_total'])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:

if __name__ == "__main__":  # Execution starts here
    start()
    freeze_panes_and_apply_header_formats()
    writer.save()

McDonald's Deutschland LLC


St. Nr. /UStLONE. 143/311/00138


BIT /ED19


TNNEN TOTAL 7.99


                              0  ... Invoice Total
0  mcdonalds_20190428_page-0001  ...          7.99
1                           NaN  ...           NaN

[2 rows x 5 columns]
EUR 993,98




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


ANSON'S Herrenhaus KG


190114117132805


                           0  ... Invoice Total
0  ansons_20190119_page-0001  ...        993,98
1                        NaN  ...           NaN

[2 rows x 5 columns]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


‘OTAL 215.00


INVOICE


[Your Company Name]


             0 Invoice Number Invoice Date        Vendor Name Invoice Total
0  Invoice_107              -              Your Company Name        215.00
1          NaN            NaN          NaN                NaN           NaN


...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


Invoice # IN-001
» data. CA OY


Panchika Randhawa


Invoice Date 29/01/2019


             0 Invoice Number Invoice Date        Vendor Name Invoice Total
0  Invoice_146   001 dataCAOY   29/01/2019  Panchika Randhawa             -
1          NaN            NaN          NaN                NaN           NaN


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


E.Breuninger GmbH & Co.


EUR 49,99


                                   0  ... Invoice Total
0  breuninger-20190330_001_page-0001  ...         49,99
1                                NaN  ...           NaN

[2 rows x 5 columns]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  
